In [1]:
!cd '/content/drive/MyDrive/Colab Notebooks'

In [2]:
!apt-get install swig -y
!pip install Cpython numpy
!pip install auto-sklearn

Reading package lists... Done
Building dependency tree       
Reading state information... Done
swig is already the newest version (3.0.12-1).
0 upgraded, 0 newly installed, 0 to remove and 15 not upgraded.


In [3]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [4]:
import autosklearn.classification

In [5]:
df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/titanic_train.csv")
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [7]:
df.drop(["PassengerId", "Name", "Ticket"], axis=1, inplace=True)

In [9]:
df["Sex"] = df["Sex"].map({"male":1, "female":0})

df["Embarked"].fillna('S', inplace=True)


In [10]:
import random
ls = []
md = df["Age"].median()
mo = df["Age"].mode()
max = df["Age"].max()

a = int((max-mo)/2)
b = int((max+mo)/2)

for i in range(177):
  ls.append(random.randint(a,b))

df.loc[np.isnan(df['Age']), 'Age'] = ls
df['Age'] = df['Age'].astype(int)

INFO:numexpr.utils:NumExpr defaulting to 2 threads.


In [11]:
df["Has_Cabin"] = df["Cabin"].apply(lambda x: 0 if type(x)==float else 1)
df.drop(["Cabin"], axis=1, inplace=True)
df

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,Has_Cabin
0,0,3,1,22,1,0,7.2500,S,0
1,1,1,0,38,1,0,71.2833,C,1
2,1,3,0,26,0,0,7.9250,S,0
3,1,1,0,35,1,0,53.1000,S,1
4,0,3,1,35,0,0,8.0500,S,0
...,...,...,...,...,...,...,...,...,...
886,0,2,1,27,0,0,13.0000,S,0
887,1,1,0,19,0,0,30.0000,S,1
888,0,3,0,50,1,2,23.4500,S,0
889,1,1,1,26,0,0,30.0000,C,1


In [12]:
embarks = pd.get_dummies(df["Embarked"], prefix="Embarked", drop_first=True)
data = pd.concat([df, embarks], axis=1)
data.drop(["Embarked"], axis=1, inplace=True)
data

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Has_Cabin,Embarked_Q,Embarked_S
0,0,3,1,22,1,0,7.2500,0,0,1
1,1,1,0,38,1,0,71.2833,1,0,0
2,1,3,0,26,0,0,7.9250,0,0,1
3,1,1,0,35,1,0,53.1000,1,0,1
4,0,3,1,35,0,0,8.0500,0,0,1
...,...,...,...,...,...,...,...,...,...,...
886,0,2,1,27,0,0,13.0000,0,0,1
887,1,1,0,19,0,0,30.0000,1,0,1
888,0,3,0,50,1,2,23.4500,0,0,1
889,1,1,1,26,0,0,30.0000,1,0,0


In [13]:
x = data.drop(["Survived"], axis=1)
y = data["Survived"]
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=57)

In [15]:
automl = autosklearn.classification.AutoSklearnClassifier(
          time_left_for_this_task=120,
          per_run_time_limit=30,
          disable_evaluator_output=False,
          resampling_strategy='holdout',
          resampling_strategy_arguments={'train_size': 0.80}
          )

automl.fit(x_train, y_train, dataset_name='titanic_dataset')

y_hat = automl.predict(x_test)
test_acc = accuracy_score(y_test, y_hat)
print("Test Accuracy score: {0}".format(test_acc))

Test Accuracy score: 0.8324022346368715
